In [56]:
###based loosly on 
##1.https://www.nltk.org/_modules/nltk/tree.html
##2.https://spacy.io/usage/linguistic-features

##Based losely on https://arxiv.org/abs/1611.01603
#https://github.com/allenai/bi-att-flow
#https://www.aclweb.org/anthology/D17-1151
#https://www.oreilly.com/learning/capturing-semantic-meanings-using-deep-learning
#https://cs224d.stanford.edu/reports/StrohMathur.pdf
#https://machinelearningmastery.com/develop-word-based-neural-language-models-python-keras/
#https://github.com/keras-team/keras/issues/4962
#https://www.nltk.org/_modules/nltk/tree.html
#https://spacy.io/usage/linguistic-features#section-sbd
#https://github.com/facebookresearch/InferSent
#https://rajpurkar.github.io/mlx/qa-and-squad/
#https://github.com/aswalin/SQuAD
#https://gist.github.com/mbollmann/ccc735366221e4dba9f89d2aab86da1e
#https://github.com/abisee/pointer-generator

import numpy as np
import pandas as pd
import ast 
from sklearn import linear_model
from sklearn import metrics
from sklearn.cross_validation import train_test_split
from nltk import Tree
from nltk.stem.lancaster import LancasterStemmer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import json
import numpy as np
import pandas as pd
import keras
import torch
from scipy import spatial

In [3]:
from random import random
from numpy import array
from numpy import cumsum
from matplotlib import pyplot
from pandas import DataFrame
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense, Flatten
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Activation
from keras.engine.input_layer import Input
from keras.layers import RepeatVector
from keras.layers import merge

from keras.models import Model 

from keras.utils import to_categorical

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

from attention_decoder import AttentionDecoder
from attention_decoder import _time_distributed_dense
import os

from textblob import TextBlob


import warnings
warnings.filterwarnings('ignore')


import pickle

import spacy
from nltk import Tree
english = spacy.load('en')

In [4]:
path = ['data','paragraphs','qas','answers']

In [44]:
for sent in english('I am here.').sents:
    print(sent.root)

am


In [5]:
curr_dir = os.getcwd()

global original
with open(curr_dir+'/train-v1.1.json', 'r') as read_training:
    original = json.load(read_training)

train_data = pd.read_json(curr_dir+"/train-v1.1.json")
valid_data = pd.read_json(curr_dir+"/dev-v1.1.json")
train_data.head(3)

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1


In [6]:


js = pd.io.json.json_normalize(json.loads(open(curr_dir+'/train-v1.1.json').read()),path)
m = pd.io.json.json_normalize(json.loads(open(curr_dir+'/train-v1.1.json').read()),path[:-1])
r = pd.io.json.json_normalize(json.loads(open(curr_dir+'/train-v1.1.json').read()),path[:-2])

In [7]:
idx = np.repeat(r['context'].values, r.qas.str.len())
ndx  = np.repeat(m['id'].values,m['answers'].str.len())

In [8]:
m['context'] = idx
js['q_idx'] = ndx

In [9]:
type(original)
#print([item for item in original.items()][:1])
d = [item for item in original.items()]
len(d)
type(d[0])

tuple

In [10]:
train_data.iloc[0,0]['paragraphs'][0]

{'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'qas': [{'answers': [{'answer_start': 515,
     'text': 'Saint Bernadette Soubirous'}],
   'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
   'id': '5733be284776f41900661182'},
  {'answers': [{'answer_start': 188, 'text': 'a copper statue of Christ

In [11]:
main = pd.concat([m[['id','question', 'context']].set_index('id'),js.set_index('q_idx')],1).reset_index()
main['c_id'] = main['context'].factorize()[0]

In [12]:
main = main.iloc[:1000,:] # chunk of 1000 there are 92000 in total 
main.shape

(1000, 6)

In [13]:
main.to_csv("train.csv", index = None)

In [14]:
paras = list(main["context"].drop_duplicates().reset_index(drop= True))

In [15]:
len(paras)

119

In [16]:
blob = TextBlob(" ".join(paras))
sent = [item.raw for item in blob.sentences]
#Documents⁩ ▸ ⁨UNCC⁩ ▸ ⁨ITCS5111_introNLP⁩ ▸ ⁨project⁩ ▸ ⁨InferSent⁩ ▸ ⁨encoder⁩

In [16]:
infersent = torch.load('InferSent/encoder/infersent.allnli.pickle', map_location=lambda storage, loc: storage)
infersent.set_glove_path("InferSent/dataset/GloVe/glove.840B.300d.txt")
infersent.build_vocab(sent, tokenize=True)

Found 4048(/4133) words with glove vectors
Vocab size : 4048


In [17]:
def get_dict_embeddings(embeddings, sent):
    
    for i in range(len(sent)):
        if i%1==0:
            print(i)
        embeddings[sent[i]] = infersent.encode([sent[i]], tokenize=True)
    return embeddings

In [18]:
questions = list(main["question"])

In [19]:
#this is a dictionary to be added to pandas dataframe
embeddings = {}
get_dict_embeddings(embeddings, sent)
get_dict_embeddings(embeddings, questions)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

{'Architecturally, the school has a Catholic character.': array([[ 0.05519995,  0.0501314 ,  0.04787038, ...,  0.00821209,
         -0.03642814,  0.04468499]], dtype=float32),
 "Atop the Main Building's gold dome is a golden statue of the Virgin Mary.": array([[ 0.07475326,  0.11794456,  0.06240866, ...,  0.01915887,
         -0.02436746,  0.10806957]], dtype=float32),
 'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".': array([[0.11262652, 0.11146843, 0.14750297, ..., 0.00293286, 0.03322018,
         0.06657629]], dtype=float32),
 'Next to the Main Building is the Basilica of the Sacred Heart.': array([[ 0.08010551,  0.11775323,  0.02186235, ...,  0.01656766,
         -0.01024125,  0.04706632]], dtype=float32),
 'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.': array([[ 0.04149358,  0.07033059,  0.03724371, ...,  0.01096804,
         -0.02892283,  0.04

In [20]:
embeddings['Next to the Main Building is the Basilica of the Sacred Heart.'][0]

array([ 0.08010551,  0.11775323,  0.02186235, ...,  0.01656766,
       -0.01024125,  0.04706632], dtype=float32)

In [21]:
even1 = {key:embeddings[key] for i, key in enumerate(embeddings) if i % 2 == 0}
odd2 = {key:embeddings[key] for i, key in enumerate(embeddings) if i % 2 == 1}

In [22]:
with open('embeddings1.pickle', 'wb') as handle:
    pickle.dump(even1, handle)

In [23]:
with open('embeddings2.pickle', 'wb') as handle:
    pickle.dump(odd2, handle)
    
del embeddings

---

In [17]:
with open("embeddings1.pickle", "rb") as f:
    even1 = pickle.load(f)
    
with open("embeddings2.pickle", "rb") as f:
    odd1 = pickle.load(f)

In [18]:
embeddings = dict(even1)
embeddings.update(odd1)
del even1, odd1


In [19]:
train_data = pd.read_csv("train.csv")
train_data.dropna(inplace=True)

In [20]:
def get_index_text(dictionary):
    index = -1
    for i in range(len(dictionary["sentences"])):
        if dictionary["text"] in dictionary["sentences"][i]: index = i
    return index

In [21]:
def process_dataframe(train_data):
    
    train_data['sentences'] = train_data['context'].apply(lambda x: [item.raw for item in TextBlob(x).sentences])
    train_data["target"] = train_data.apply(get_index_text, axis = 1)
    train_data['sent_emb'] = train_data['sentences'].apply(lambda x: [embeddings[item][0] if item in\
                                                           embeddings else np.zeros(4096) for item in x])
    train_data['quest_emb'] = train_data['question'].apply(lambda x: embeddings[x] if x in embeddings else np.zeros(4096) )
    return train_data   

In [22]:
train_data = process_dataframe(train_data)
train_data.head(3)

,index,question,context,answer_start,text,c_id,sentences,target,sent_emb,quest_emb
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0,"[Architecturally, the school has a Catholic ch...",5,"[[0.055199955, 0.050131403, 0.047870383, 0.016...","[[0.10956341, 0.1142294, 0.044289418, 0.053171..."
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0,"[Architecturally, the school has a Catholic ch...",2,"[[0.055199955, 0.050131403, 0.047870383, 0.016...","[[0.10951652, 0.11030623, 0.05210007, 0.030539..."
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0,"[Architecturally, the school has a Catholic ch...",3,"[[0.055199955, 0.050131403, 0.047870383, 0.016...","[[0.002622683, 0.14930709, 0.030639844, 0.0624..."


In [23]:
def cos_similarity(x):
    my_list= []
    for item in x["sent_emb"]:
        my_list.append(spatial.distance.cosine(item,x["quest_emb"][0]))
    return my_list 

In [24]:
def predict_index(distances):
    return np.argmin(distances)   

In [25]:
def predictions(train_data):
    
    train_data["cos_similarity"] = train_data.apply(cos_similarity, axis = 1)
    train_data["diff"] = (train_data["quest_emb"] - train_data["sent_emb"])**2
    train_data["euclidean_dis"] = train_data["diff"].apply(lambda x: list(np.sum(x, axis = 1)))
    del train_data["diff"]
    
    print("cosine start")
    
    train_data["pred_index_cosine"] = train_data["cos_similarity"].apply(lambda x: predict_index(x))
    train_data["pred_index_euclidian"] = train_data["euclidean_dis"].apply(lambda x: predict_index(x))
    
    return train_data

In [26]:
predicted_cos_eucl = predictions(train_data)

cosine start


In [27]:
predicted_cos_eucl.head(2)

,index,question,context,answer_start,text,c_id,sentences,target,sent_emb,quest_emb,cos_similarity,euclidean_dis,pred_index_cosine,pred_index_euclidian
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0,"[Architecturally, the school has a Catholic ch...",5,"[[0.055199955, 0.050131403, 0.047870383, 0.016...","[[0.10956341, 0.1142294, 0.044289418, 0.053171...","[0.43262481689453125, 0.3686484098434448, 0.36...","[14.108409, 15.0115595, 18.043663, 13.160973, ...",5,5
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0,"[Architecturally, the school has a Catholic ch...",2,"[[0.055199955, 0.050131403, 0.047870383, 0.016...","[[0.10951652, 0.11030623, 0.05210007, 0.030539...","[0.45407456159591675, 0.32262009382247925, 0.3...","[12.889506, 12.285218, 16.843704, 8.361172, 11...",3,3


In [28]:
predicted_cos_eucl["cos_similarity"][0]

[0.43262481689453125,
 0.3686484098434448,
 0.3676990270614624,
 0.3795006275177002,
 0.3742116093635559,
 0.19309097528457642,
 0.3657878041267395]

In [29]:
predicted_cos_eucl["euclidean_dis"][0]

[14.108409, 15.0115595, 18.043663, 13.160973, 12.855492, 9.627245, 15.943851]

In [30]:
#defining accuracy formula
def accur(target, predicted_cos_eucl): 
    acc = (target==predicted_cos_eucl).sum()/len(target)
    return acc

In [31]:
print(accur(predicted_cos_eucl["target"], predicted_cos_eucl["pred_index_euclidian"]))

0.388


In [32]:
print(accur(predicted_cos_eucl["target"], predicted_cos_eucl["pred_index_cosine"]))

0.608


In [33]:
predicted_cos_eucl.to_csv("train_sentence_detection1.csv", index=None)

In [34]:
doc = english(predicted_cos_eucl.iloc[0,1])

In [35]:
predicted_cos_eucl.iloc[0,1]

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [36]:
predicted_cos_eucl.iloc[0,2]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [37]:
def to_nltk_tree(node):
    if node.n_lefts + node.n_rights > 0:
        return Tree(node.orth_, [to_nltk_tree(child) for child in node.children])
    else:
        return node.orth_

In [55]:
for sent in english(predicted_cos_eucl.iloc[0,2]).sents:
    print(sent.root)
    print(sent.root.children)
    print(sent.noun_chunks)
    

has
is
is
is
is
is
is


In [45]:
#paragraph
[to_nltk_tree(sent.root).pretty_print()  for sent in english(predicted_cos_eucl.iloc[0,2]).sents]

                    has                              
        _____________|_________________               
       |         |   |  school     character         
       |         |   |    |      ______|________      
Architecturally  ,   .   the    a            Catholic

                            is                           
  __________________________|________                     
 |           Atop                  statue                
 |            |                  ____|_________           
 |           dome               |    |         of        
 |    ________|______           |    |         |          
 |   |            Building      |    |        Mary       
 |   |     __________|______    |    |      ___|_____     
 .  gold the        Main    's  a  golden the      Virgin

                                                  is                                                   
  ________________________________________________|______________________                       

[None, None, None, None, None, None, None]

In [46]:
st = LancasterStemmer()

In [47]:
#question
[to_nltk_tree(sent.root).pretty_print()  for sent in english(predicted_cos_eucl.iloc[0,1]).sents]

                  appear                             
  __________________|____________________________     
 |      |      |    |         |           |      in  
 |      |      |    |         |           |      |    
 |      |      |    To       Mary         in   France
 |      |      |    |      ___|_____      |      |    
did allegedly  ?   whom  the      Virgin 1858 Lourdes



[None]

In [61]:
def match_roots(x):
    question = x["question"].lower()
    sentences = english(x["context"].lower()).sents
    
    question_root = st.stem(str([sent.root for sent in english(question).sents][0]))
    
    li = []
    for i,sent in enumerate(sentences):
        roots = [st.stem(chunk.root.head.text.lower()) for chunk in sent.noun_chunks]

        if question_root in roots: 
            for k,item in enumerate(ast.literal_eval(x["sentences"])):
                if str(sent) in item.lower(): 
                    li.append(k)
    return li

In [62]:
predicted_cos_eucl.columns
predicted_cos_eucl["question"][0]

'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?'

In [63]:
#predicted = predicted.drop("index", axis=1)
#predicted = predicted.drop("c_id", axis=1)
#type(predicted)
predicted_cos_eucl.head(3)

,index,question,context,answer_start,text,c_id,sentences,target,sent_emb,quest_emb,cos_similarity,euclidean_dis,pred_index_cosine,pred_index_euclidian
0,5733be284776f41900661182,To whom did the Virgin Mary allegedly appear i...,"Architecturally, the school has a Catholic cha...",515,Saint Bernadette Soubirous,0,"[Architecturally, the school has a Catholic ch...",5,"[[0.055199955, 0.050131403, 0.047870383, 0.016...","[[0.10956341, 0.1142294, 0.044289418, 0.053171...","[0.43262481689453125, 0.3686484098434448, 0.36...","[14.108409, 15.0115595, 18.043663, 13.160973, ...",5,5
1,5733be284776f4190066117f,What is in front of the Notre Dame Main Building?,"Architecturally, the school has a Catholic cha...",188,a copper statue of Christ,0,"[Architecturally, the school has a Catholic ch...",2,"[[0.055199955, 0.050131403, 0.047870383, 0.016...","[[0.10951652, 0.11030623, 0.05210007, 0.030539...","[0.45407456159591675, 0.32262009382247925, 0.3...","[12.889506, 12.285218, 16.843704, 8.361172, 11...",3,3
2,5733be284776f41900661180,The Basilica of the Sacred heart at Notre Dame...,"Architecturally, the school has a Catholic cha...",279,the Main Building,0,"[Architecturally, the school has a Catholic ch...",3,"[[0.055199955, 0.050131403, 0.047870383, 0.016...","[[0.002622683, 0.14930709, 0.030639844, 0.0624...","[0.39596283435821533, 0.315723180770874, 0.346...","[11.590836, 12.180486, 16.56459, 7.4446793, 9....",3,3


In [64]:
predicted_cos_eucl["context"][0]

'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.'

In [65]:
predicted_cos_eucl["root_match_idx"] = predicted_cos_eucl.apply(match_roots, axis = 1)

ValueError: ('malformed node or string: [\'Architecturally, the school has a Catholic character.\', "Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary.", \'Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".\', \'Next to the Main Building is the Basilica of the Sacred Heart.\', \'Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.\', \'It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.\', \'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.\']', 'occurred at index 0')

In [ ]:
predicted_cos_eucl["root_match_idx_first"]= predicted_cos_eucl["root_match_idx"].apply(lambda x: x[0] if len(x)>0 else 0)

In [ ]:
predicted_cos_eucl.to_csv("train_detect_sent1.csv", index=None)